In [3]:
import pandas as pd
import numpy as np
import folium 
import requests
import json
import time
import seaborn as sns
import matplotlib.pyplot as plt
from math import *
import random
from tqdm import tqdm
from iteration_utilities import deepflatten
from scrape import *
import datetime as dt

pd.options.display.max_columns = 100

In [7]:
valid_ids = pd.read_csv("data/valid_ids_05_02_2023.csv")
df = extract(URL, valid_ids.iloc[:,0].to_list())
df = transform(df)
df.to_parquet("data/scrape_2023.parquet", index=False)
df

100%|██████████████████████████████████████████████████████████████████████████████████| 473/473 [02:19<00:00,  3.40it/s]


,listing_id,available_at,months_available_for,address,unit_number,city,zip,contact_preference,contact_name,short_description,rent_amount,rent_type,pets_allowed,gender_preference,num_bedrooms,num_bathrooms,housing_type,gmap_latitude,gmap_longitude,expires_at,listing_within_city,created_at,listing_photos,overall_safety_rating,meets_minimum_requirements,exceeds_requirements,has_fire_resistant_construction_type,has_gender_preference,has_female_gender_preference,has_male_gender_preference,is_single_room,amenity_air_conditioning,includes_electricity,electronic_payments_allowed,includes_furniture,includes_heat,includes_internet,amenity_kitchen_available,amenity_laundry_available,is_near_bus,amenity_offstreet_parking_available,includes_offstreet_parking,psparking_available,includes_water,market_grad,market_faculty,market_ugrad,market_sabbatical,market_fall_sublet,market_spring_sublet,market_summer_sublet,market_short_term,distance_to_campus,price_per_room,price_per_unit,year,month,uuid
0,20,2023-08-01 00:00:00+00:00,12,219 Kelvin Place,1,Ithaca,14850,EmailOnly,Kathleen Cacciotti,Furnished Rooms in Large North Campus Area Hou...,795.0,Price Per Unit,0,No Preference,1.0,3.0,Room to Rent,42.456514,-76.483689,2023-06-19T00:00:00.000Z,No,2021-11-29 15:19:58.835000+00:00,"[{'PhotoId': 59, 'PhotoName': '00w0w_czyTl5rfW...",NaN,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,0,0,0,991.667090,795.0,795.0,2023,8,20-2023-8
1,23,2023-08-01 00:00:00+00:00,12,116 LAKE STREET,,Ithaca,14850,EmailOnly,Manager,One Bedroom Furnished. Very high quality. We...,1590.0,Price Per Unit,0,No Preference,1.0,1.0,Rent,42.450724,-76.492924,2023-04-27T00:00:00.000Z,Yes,2021-11-29 15:19:58.917000+00:00,"[{'PhotoId': 11992, 'PhotoName': 'P1230702.JPG...",12.0,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,1,1,1,0,1,0,0,0,0,737.144681,1590.0,1590.0,2023,8,23-2023-8
2,39,2023-08-05 00:00:00+00:00,12,232 LINDEN AVENUE,None,Ithaca,14850,BothEmailAndPhone,IKE NESTOPOULOS,Visit our website demosjohnnycollegetownrental...,1050.0,Price Per Person,0,No Preference,5.0,2.0,Rent,42.440776,-76.484404,2022-09-24T00:00:00.000Z,Yes,2021-11-29 15:19:59.363000+00:00,"[{'PhotoId': 27396, 'PhotoName': '232 Linden A...",8.0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,765.302946,1050.0,5250.0,2023,8,39-2023-8
3,40,2023-06-06 00:00:00+00:00,12,232 LINDEN AVENUE,None,Ithaca,14850,BothEmailAndPhone,john puglia,Visit our website\ndemosjohnnycollegetownrenta...,1100.0,Price Per Person,0,No Preference,2.0,2.0,Rent,42.440884,-76.484107,2022-09-24T00:00:00.000Z,Yes,2021-11-29 15:19:59.398000+00:00,"[{'PhotoId': 124, 'PhotoName': '232LindenOutsi...",8.0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,755.192456,1100.0,2200.0,2023,6,40-2023-6
4,76,2023-08-10 00:00:00+00:00,12,717 BUFFALO STREET E,None,Ithaca,14850,BothEmailAndPhone,Sharon Huang,Furnished Rooms For a single person or groups....,715.0,Price Per Person,0,No Preference,10.0,4.0,Room to Rent,42.441320,-76.487432,2023-06-25T00:00:00.000Z,Yes,2021-11-29 15:20:01.151000+00:00,"[{'PhotoId': 267, 'PhotoName': 'House.jpg', 'I...",8.0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,0,0,0,731.930278,715.0,7150.0,2023,8,76-2023-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,4280,2023-05-22 00:00:00+00:00,2,112-14 Ferris Place,114,Ithaca,14850,BothEmailAndPhone,Eric Yen,"6 Bedrooms, 2 Baths, Living Room, Kitchen, Was...",800.0,Price Per Person,0,No Preference,6.0,2.0,Room to Rent,42.438424,-76.489429,2023-07-28T00:00:00.000Z,Yes,2023-04-27 18:54:33.206000+00:00,"[{'PhotoId': 30491, 'PhotoName': '112+ferris+2...",11.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,1,0,1088.868982,800.0,4800.0,2023,5,4280-2023-5
468,4281,2023-12-23 00:00:00+00:00,7,210 PENNSYLVANIA AVE,None,Ithaca,14850,BothEmailAndPhone,Jake Stebbins,1 Private Bedroom a

In [6]:
data2021 = pd.read_csv("data/raw_scrape_oct_25_2021.csv")
data2021 = data2021.iloc[:,1:]
data2021 = transform(data2021)
data2021.to_parquet("data/scrape_2021.parquet", index=False)
data2021

,listing_id,available_at,months_available_for,address,unit_number,city,zip,contact_preference,contact_name,short_description,rent_amount,rent_type,pets_allowed,gender_preference,num_bedrooms,num_bathrooms,housing_type,gmap_latitude,gmap_longitude,expires_at,listing_within_city,created_at,overall_safety_rating,has_fire_resistant_construction_type,exceeds_requirements,meets_minimum_requirements,has_gender_preference,has_female_gender_preference,has_male_gender_preference,is_single_room,amenity_air_conditioning,includes_electricity,electronic_payments_allowed,includes_furniture,includes_heat,includes_internet,amenity_kitchen_available,amenity_laundry_available,is_near_bus,amenity_offstreet_parking_available,includes_offstreet_parking,psparking_available,includes_water,market_grad,market_faculty,market_ugrad,market_sabbatical,market_fall_sublet,market_spring_sublet,market_summer_sublet,market_short_term,distance_to_campus,price_per_room,price_per_unit,year,month,uuid
1,2,2018-08-14 04:00:00+00:00,5,212-12.5 DELAWARE AVENUE,NaN,Ithaca,14850,Phone Only,Bill Cassaniti,SPRING 2018 SEMESTER avail: 2 BRs + kitch & co...,975.0,Price per Person,0,No Preference,2.0,2.0,Rent,42.440071,-76.483517,2017-11-20T05:00:00.000Z,Yes,2017-03-13 23:00:17+00:00,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,850.655237,975.000000,1950.0,2018,8,2-2018-8
3,4,2019-08-15 04:00:00+00:00,10,219 Eastern Heights Drive,3,Ithaca,14850,Email Only,Chad Coates,Graduate Housing,715.0,Price per Unit,0,Male,1.0,1.0,Rent,42.422104,-76.449149,2019-08-15T04:00:00.000Z,No,2017-03-14 16:43:40+00:00,NaN,0,0,0,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,4087.414219,715.000000,715.0,2019,8,4-2019-8
4,5,2021-06-24 04:00:00+00:00,12,216 Dearborn Pl,NaN,Ithaca,14850,Both Email and Phone,Terri Hoppenrath,216 Dearborn - Graduate Student Cooperative,875.0,Price per Person,0,No Preference,1.0,3.5,Room to Rent,42.456552,-76.482074,2021-07-20T04:00:00.000Z,No,2017-03-14 22:35:20+00:00,NaN,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,1018.337167,875.000000,875.0,2021,6,5-2021-6
5,6,2022-08-01 04:00:00+00:00,12,203 PROSPECT STREET,NaN,Ithaca,14850,Both Email and Phone,Caroline Rasmussen,Six bedroom house - great location - 2 kitchen...,700.0,Price per Person,1,No Preference,6.0,2.0,Rent,42.437401,-76.494222,2021-11-29T05:00:00.000Z,Yes,2017-03-15 13:00:16+00:00,8.0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1369.102514,700.000000,4200.0,2022,8,6-2022-8
6,8,2017-08-01 04:00:00+00:00,12,203 PROSPECT STREET,A,Ithaca,14850,Both Email and Phone,Caroline Rasmussen,$716 Landlord pays Utilities! 1 block from It...,716.0,Price per Person,1,No Preference,3.0,1.0,Rent,42.437342,-76.494235,2017-05-10T04:00:00.000Z,Yes,2017-03-15 13:00:16+00:00,8.0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1375.155058,716.000000,2148.0,2017,8,8-2017-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,3610,2021-10-22 04:00:00+00:00,12,"104 Brandywine Dr Ithaca, NY 14850",NaN,Ithaca,14850,Both Email and Phone,Ann Wang,Unique & Desirable Home for Your Family & Chil...,2950.0,Price per Unit,1,No Preference,3.0,2.0,Rent,42.473067,-76.470746,2021-11-22T05:00:00.000Z,No,2017-05-10 19:03:55+00:00,NaN,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,3057.952748,983.333333,2950.0,2021,10,3610-2021-10
3553,3611,2022-06-05 04:00:00+00:00,12,708 STEWART AVENUE,3,Ithaca,14850,Both Email and Phone,Shane Varrichio,Great Modern 2 Br Apt Next To Libe Slope,925.0,Price per Person,0,No Preference,2.0,1.0,Rent,42.450026,-76.490512,NaN,Yes,2017-06-14 13:43:57+00:00,15.0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,1,1,1,0,0,0,0,0,526.329509,925.000000,1850.0,2022,6,3611-2022-6
3554,3612,2021-12-18 05:00:00+00:00,6,320 DRYDEN ROAD,1B,Ithaca,14850,Both Email and Phone,Marielle Zweig,5 Bedroom 2 Bath

In [16]:
def generate_map(df=df, gmap_latitude=42.44764589643703, gmap_longitude=-76.48496728637923):
    """
    param: df
    type: pd.DataFrame() Object
    param gmapLatitude: 
    type df.gmapLatitude: float 
    param gmapLongitude: 
    type df.gmapLongitude: float
    rtype: folium.Map() Object
    """
    res = folium.folium.Map(location=[gmap_latitude, gmap_longitude],width='100%', height='100%', left='0%', top='0%',
                     zoom_start=14,position='relative', tiles='OpenStreetMap')
    folium.vector_layers.CircleMarker([gmap_latitude, gmap_longitude], radius=2.5, color='red', opacity=1).add_to(res)
    for index, row in df.iterrows(): 
        folium.vector_layers.CircleMarker([row["gmap_latitude"], row["gmap_longitude"]], radius=0.2, color="blue", opacity=0.5).add_to(res)
    folium.vector_layers.Circle(location=[gmap_latitude, gmap_longitude],radius=convert_meters_to_feet(2.5,prefix="kilo", unit_from="meters",
        unit_to="feet",output_type=float),color="white").add_to(res)
    return res


In [17]:
generate_map(df)

In [18]:
generate_map(data2021)